In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
amazon_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/amazon_data.csv')

In [16]:
amazon_df.shape

(20000, 2)

**pre-processing**

In [17]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=56ae46da0ac687ec9eaa56fac20c9b657c4bd09cc9442612a540b08aefbef29f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [18]:
pip install nltk

In [19]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Needed for WordNetLemmatizer

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [20]:
# Ensure consistency in language detection results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

def clean_text(review):
    # Remove URLs
    review = re.sub(r'http\S+|www\S+|https\S+', '', review, flags=re.MULTILINE)
    # Remove HTML tags
    review = re.sub(r'<.*?>', '', review)
    # Remove special characters and digits
    review = re.sub(r'\d+', '', review)
    review = re.sub(r'[^A-Za-z\s]+', '', review)  # Keep only alphabets and spaces
    return review

def remove_stopwords(review):
    return ' '.join([word for word in review.split() if word not in stop_words])

def lemmatize_text(review):
    return ' '.join([lemmatizer.lemmatize(word) for word in review.split()])

def preprocess_review(review):
    if isinstance(review, str):  # Ensure the review is a valid string
        # Detect language, and only process if it's English
        if detect_language(review) != 'en':
            return None
        # Apply cleaning, stopword removal, and lemmatization
        review = clean_text(review)
        review = review.lower()
        review = remove_stopwords(review)
        review = lemmatize_text(review)
        return review.strip() if len(review.strip()) > 0 else None
    return None

In [21]:
amazon_df['Cleaned_Reviews'] = amazon_df['Review'].apply(preprocess_review)

# Remove rows where Cleaned_Reviews is None (indicating non-English or empty reviews)
amazon_df = amazon_df.dropna(subset=['Cleaned_Reviews'])

In [ ]:
amazon_df["Review", "Cleaned_Reviews"].shape

(16695, 2)

In [ ]:
amazon_df["Review", "Cleaned_Reviews"].head()

,Review,Cleaned_Reviews
0,Pros:\n- Pairs and switches well with multiple...,pro pair switch well multiple device android p...
1,Sound quality is okay for this price. They are...,sound quality okay price light comfortable sma...
2,"Easy to operate, good product quality, very sa...",easy operate good product quality satisfied
4,I was hoping for it to blow my head off but it...,hoping blow head still attached pretty quiet e...
5,"Despite not being Apple airpods, these earbuds...",despite apple airpods earbuds trick responsive...


In [25]:
def map_to_sentiment(rating):
      if rating <= 2:
          return 0
      elif 2 < rating < 4:
          return 1
      else:
           return 2

amazon_df["Sentiment"] = amazon_df["Rating"].apply(map_to_sentiment)

In [27]:
amazon_df[["Cleaned_Reviews", "Sentiment"]].head()

,Cleaned_Reviews,Sentiment
0,model may ok sedentary type im active get arou...,0
1,fast read filled unexpected humour profound in...,2
2,bought one chargersthe instruction say light s...,0
3,excited find book ostensibly muslim feminism v...,0
4,big jvc fan like model suspiscious saw several...,0
